<div align="center">
    <h2> CSCI7000-011 Homework Assignment 5</h2>
</div>

* There are 2 problems with 4 points each. Total points = 8.
* Before you turn it in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).
* Fill in any place that says `YOUR CODE HERE`.
* Do not tamper with testcases.

In [1]:
(* Required definitions *)
let asrt = function
 | (true,_) -> ()
 | (false, str) -> failwith ("Assertion failure: "^str)

val asrt : bool * string -> unit = <fun>


In [7]:
(* Abstract Syntax *)

(* CoreML type *)

(* base types *)
type base_typ =
  | TyInt
  | TyBool
  
type typ = 
  | TyRef of string * base_typ * expr (* "refined" type. e.g., {v: int | v>0} *)
  | TyArr of string * typ * typ (* Dependent arrow type. e.g., x:{v0: int | ¬(v0=0)} -> {v1: int | v1 > x} *)

and expr =
  | Var of string
  | App of expr * expr
  | Lam of string * typ * typ option * expr (* \(x:T1):T2.e or \(x:T1).e *)
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Gt of expr * expr
  | Lt of expr * expr
  | Not of expr
  | And of expr * expr
  | Or of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * typ * expr * expr (* let rec (x:poly_typ) = e_1 in e_2 *)

type base_typ = TyInt | TyBool


type typ = TyRef of string * base_typ * expr | TyArr of string * typ * typ
and expr =
    Var of string
  | App of expr * expr
  | Lam of string * typ * expr
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Gt of expr * expr
  | Lt of expr * expr
  | Not of expr
  | And of expr * expr
  | Or of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * typ * expr * expr


In [10]:
exception TypeError of string
type typbind = string * typ
type tyenv = typbind list

exception TypeError of string


type typbind = string * typ


type tyenv = typbind list


In [11]:
let rec is_subtyp env t1 t2 =
  failwith "Not impl."

val asrt_subtyp : 'a -> 'b -> 'c -> 'd = <fun>


In [12]:
let rec subst (e,x) t = 
  failwith "Not impl."

val subst : 'a * 'b -> 'c -> 'd = <fun>


In [13]:
let trivial (t:base_typ) : typ = TyRef("v",t, BoolConst true)

val trivial : base_typ -> typ = <fun>


In [ ]:
let rec type_check (env: tyenv) (e: expr) : typ = match e with
  | Var x -> (try List.assoc x env 
              with Not_found -> raise @@ TypeError ("Variable "^x^" undefined"))
  | Lam(x,ty1,ty2op,e1) -> 
    let env' = (x,ty)::env in
    let bdy_typ = type_check env' e1 in
    (match ty2op with 
      | Some ty2 when is_subtype env bdy_typ ty2 -> ty2
      | Some _ -> raise @@ TypeError "Subtyping check failed"
      | None -> bdy_typ)
  | App(e1,e2) -> 
    let ty1 = type_check env e1 in
    let ty2 = type_check env e2 in
    begin
      match ty1 with
      | TyArr (x, t1, t2) when is_subtype env ty1 t1 -> subst (e2,x) t2
      | TyArr (x, t1, t2) -> raise @@ TypeError "Subtyping check failed"
      | _ -> raise @@ TypeError "Left hand side of application is not a function type" 
    end
  | IntConst _ -> trivial TyInt
  | BoolConst _ -> trivial TyBool
  | Plus(e1,e2) | Minus (e1,e2) | Mult(e1,e2) ->
    let (ty1,ty2) = (type_check env e1, type_check env e2) in
    begin
      match ty1,ty2 with
      | TyInt, TyInt -> TyInt
      | _ -> raise @@ TypeError "Arithmetic expression needs to have Int type"
    end
  | Eq (e1,e2) ->
    let (ty1,ty2) = (mono_of @@ type_check env e1, mono_of @@ type_check env e2) in
    if ty1 = ty2 then mono TyBool 
    else raise @@ TypeError "Expressions being compared have different types"
  | Ite(e1,e2,e3) -> 
    let ty1 = mono_of @@ type_check env e1 in
    let ty2 = mono_of @@ type_check env e2 in
    let ty3 = mono_of @@ type_check env e3 in
    (match ty1 with
    | TyBool when ty2 = ty3 -> mono ty2
    | TyBool -> raise @@ TypeError "If and then branches need to have same type"
    | _ -> raise @@ TypeError "If scrutinee has non-boolean type")
  | Let(x,e1,e2) -> 
    let pty1 = type_check env e1 in
    let pty2 = type_check ((x,pty1)::env) e2 in
    pty2
  | LetRec (x,ptx,e1,e2) ->
    let env' = (x,ptx)::env in
    let pty1 = type_check env' e1 in
    let _ = if ptx=pty1 then () 
        else raise @@ TypeError ("Asserted type of "^x^" not same as computed type") in
    let pty2 = type_check env' e2 in
    pty2